In [204]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [205]:
%cd gdrive/MyDrive/Adaboost/

[Errno 2] No such file or directory: 'gdrive/MyDrive/Adaboost/'
/content/gdrive/MyDrive/Adaboost


In [206]:
!pip install numpy scikit-learn pandas matplotlib

In [207]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random

In [208]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import minmax_scale, StandardScaler, MinMaxScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [209]:
dataset = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

dataset.drop(['customerID'] , inplace=True , axis='columns')

for i in dataset.columns:
  #print(i, dataset[i].nunique())
  if(dataset[i].nunique() == 2 and dataset[i].dtype == 'object'):
    #print(dataset[i].unique()[0], dataset[i].unique()[1])
    dataset[i] = dataset[i].map({dataset[i].unique()[0] : 0, dataset[i].unique()[1] : 1})

dataset['TotalCharges'] = pd.to_numeric(dataset['TotalCharges'], errors='coerce')

# mean = dataset['TotalCharges'].mean()
# dataset['TotalCharges'] = dataset['TotalCharges'].replace([np.nan],mean)
arr = []
for i in range(len(dataset['tenure'])):
  if(dataset.loc[i , 'tenure'] == 0):
    imagined_tenure = random.random()
    # print(imagined_tenure)
    # print(dataset.loc[i , ['MonthlyCharges']])
    the_loc = dataset.loc[i , ['MonthlyCharges']]
    # print(the_loc)
    dataset.at[i , ['TotalCharges']] = float(the_loc * imagined_tenure)
    # print(dataset.loc[i , ['TotalCharges']])
    arr.append(i)
#dataset.info()
# for i in dataset['TotalCharges']:
#   if(np.isnan(i)):
#     i.replace(mean)

dataset['Churn'] = dataset['Churn'].map({1: 1, 0: -1})
print(dataset['Churn'].unique())


dataset = pd.get_dummies(dataset)
#print(dataset.head())


def normalize(dataset):
  for i in dataset.columns:
    if(dataset[i].nunique()!=2):
      dataset[i] = (dataset[i]-dataset[i].min())/(dataset[i].max()-dataset[i].min())
  return dataset

#dataset = normalize(dataset)



dataset = dataset[3025:5050]
# dataset.head(100)

#dataset['TotalCharges'].count()

#pd.Index(dataset['Churn']).value_counts()




[-1  1]


In [210]:
# dataset.at[0, ['tenure']] = 0.2
# dataset.loc[0 , ['tenure']]
# arr = []
# for i in range(len(dataset['tenure'])):
#   if(dataset.loc[i , 'tenure'] == 0):
#     imagined_tenure = random.random()
#     dataset.at[i , ['TotalCharges']] = dataset.loc[i , ['MonthlyCharges']] * imagined_tenure
#     arr.append(i)
# dataset.loc[488 ,['TotalCharges']]


In [211]:
x = dataset.drop(['Churn'], axis='columns')

y = dataset['Churn']

# x
# y

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

x_train = normalize(x_train)
x_test = normalize(x_test)

In [212]:
def Tanh(z):
    th = np.tanh(z)
    # if(np.isnan(th)):
    #   if(z < 0):
    #     return -1
    #   else:
    #     return 1

    return float(th)

In [213]:
def hypothesistanh(theta,x):
    #print ('theta ', theta)	
    # z = 0
    # for i in range(len(theta)):
    #     xi = x[i]
    #     z += np.dot(xi,theta[i].transpose())

    z = np.dot(theta,x)
    return Tanh(z)

In [214]:
def cost_function_derivativetanh(x_train,y_train,theta,j,m,alpha):
    sumErrors = 0
    for i in range(m):
        xi = x_train.iloc[i]
        hi = hypothesistanh(theta,xi)
        yt = y_train.iloc[i]
        hit = np.subtract((float)(hi),float(yt))
        error = np.dot(hit,x_train.iloc[i][j])
        error = np.dot(error,(1-hi*hi))
        sumErrors += error
    #m = len(y_test)
    constant = float(alpha)/float(m)
    j = constant * sumErrors
    #print("SumErrors", sumErrors)
    return j

In [215]:
def gradient_descenttanh(x_train,y_train,theta,m,alpha):
    new_theta = []
    for j in range(len(theta)):
        new_theta_value = theta[j] - cost_function_derivativetanh(x_train,y_train,theta,j,m,alpha)
        new_theta.append(new_theta_value)
    #print ('theta ', new_theta)	
    return new_theta

In [216]:
def error_calc(x_train,y_train,theta):
  total = 0
  m = len(y_train)
  for i in range(m):
        xi = x_train.iloc[i]
        hi = hypothesistanh(theta,xi)
        yt = y_train.iloc[i]
        hit = np.subtract((float)(hi),float(yt))
        total += hit*hit

  return total/float(m)

In [217]:
len(x_test)

405

In [218]:
def logistic_regression_tanh(x_train,y_train,alpha,theta,num_iters):
    m = len(y_train)
    for x in range(num_iters):
        new_theta = gradient_descenttanh(x_train,y_train,theta,m,alpha)
        theta = new_theta
        #total = error_calc(theta)
        #if x % 1000 == 0:
            #cost_functiontanh(theta,m)
        #print ('theta ', theta)
        #print('total' , total)	
            #print ('cost is ', cost_functiontanh(theta,m))
    score = 0
    length = len(x_test)
    for i in range(length):
        prediction = hypothesistanh(theta,x_test.iloc[i])
        if prediction < 0:
          prediction = -1
        else:
          prediction = 1
        # if(prediction == -1):
        #   print("prediction ", prediction)
        answer = y_test.iloc[i]
        if prediction == answer:
            score += 1
    my_score = float(score) / float(length)
    print ('Your score with tanh: ', my_score)

    return theta

In [219]:
def ada_boost(num_iter):
  m = len(y_train)

  weight = np.empty(m)
  weight.fill(1.0/float(m))
  #print(weight)
  
  hypothesis_container = []
  hypothesis_weight = []

  np.random.seed(5)

  x_train_sampled = x_train
  y_train_sampled = y_train

  for i in range(num_iter):
    
    resampled_int = np.random.choice(x_train_sampled.shape[0], size=m, p=weight)
    #print(resampled_int)
    x_train_sampled = x_train_sampled.iloc[resampled_int]
    y_train_sampled = y_train_sampled.iloc[resampled_int]
    #print(temp)
    #print(x_train_sampled)
    
    theta = logistic_regression_tanh(x_train_sampled, y_train_sampled, alpha, initial_theta ,iterations)
    #hypothesis_container.append(theta)

    error = 0

    for j in range(m):
      xj = x_train_sampled.iloc[j]
      hj = hypothesistanh(theta,xj)
      if hj < 0:
        hj = -1
      else:
        hj = 1
      yj = y_train_sampled.iloc[j]
      if hj != yj:
        error = error + weight[j]

    if error > 0.5:
      print(error)
      continue

    for j in range(m):
      xj = x_train_sampled.iloc[j]
      hj = hypothesistanh(theta,xj)
      if hj < 0:
        hj = -1
      else:
        hj = 1
      yj = y_train_sampled.iloc[j]
      if hj == yj:
        weight[j] = weight[j] * float(error)/(1.0-float(error))

    weight = weight/np.sum(weight)
    #print(weight)

    hypothesis_container.append(theta)
    hypothesis_weight.append(np.log2(  (1.0-float(error))   /   float(error)   ))

  h_cap = Weighted_majority(x_test , hypothesis_container , hypothesis_weight)
  #print(h_cap)

  m2 = len(x_test)
  score = 0
  for i in range(m2):
    if h_cap[i] < 0:
      h_cap[i] = -1
    else:
      h_cap[i] = 1

    answer = y_test.iloc[i]
    if h_cap[i] == answer:
      score += 1
  print(h_cap)
  print(float(score) / float(m2))


def Weighted_majority(x , hypothesis_container , hypothesis_weight):
  m = len(x)
  iter = len(hypothesis_weight)
  #print(iter)
  h_cap = np.zeros(m)
  for k in range(iter):
    for i in range(m):
      xi = x.iloc[i]
      hi = hypothesistanh(hypothesis_container[k] , xi)
      mul = hi*hypothesis_weight[k]
      h_cap[i] = h_cap[i] + mul

  #print(h_cap)
  return h_cap
  








In [220]:
initial_theta = np.zeros(x_test.iloc[1].shape)
#x_test.iloc[1]
alpha = 0.5
iterations = 10
#logistic_regression_tanh(alpha,initial_theta,iterations)
ada_boost(5)

Your score with tanh:  0.7802469135802469
Your score with tanh:  0.6049382716049383
Your score with tanh:  0.691358024691358
Your score with tanh:  0.2617283950617284
Your score with tanh:  0.28641975308641976
[ 0.28458526 -0.26670753 -2.3203746  -2.20607321 -2.43018158  0.25584425
 -1.52260287 -0.56768495  0.69459707 -0.90514636 -0.17806645 -2.07873039
 -2.91504908  0.77113599  0.35043315 -1.49596852 -2.54870014  0.32701195
 -2.1402917  -0.02945547  1.17432068  0.58533855 -2.77422334 -1.84608269
 -0.43270851 -0.20936275 -2.13740409 -0.66264884 -2.953478   -0.28452405
 -2.37734452 -1.97391035 -2.2419813   0.48498265 -1.52869945  0.04330297
  1.26003185 -2.20407525  0.86886152  0.99197676 -0.8424172  -2.3031283
 -0.68916057 -1.84830069 -2.72009491  0.77847229  0.63828499 -0.83519447
  0.15780274 -2.01271388 -3.0208987  -1.29505271 -1.44156708  0.27142106
 -0.33787034 -0.10543592  0.59023421 -2.23894641 -0.83150747  0.63444288
 -0.43385583 -1.11335299 -1.85353801 -0.4186874  -0.05836717 

In [221]:
# bal = np.array([])
# counter = 0
# abal = [1,2,3,4,5]
# dhon = [3,4,6,7,8,9]
# bal.append(abal)
# bal.append(dhon)
# len(bal[1])
# bal = np.empty(5)
# bal.fill(1/5)
# bal
# len(y_train)
# ada_boost(3)
a = np.array([[1, 0],
              [0, 1]])

a[0]


array([1, 0])

In [222]:
a = [1,2,3,4,5]
b = [1,2,3,4,5]
c = np.dot(a,b)
c

55